In [306]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import random

In [307]:
pipe = pipeline("fill-mask", model='distilroberta-base')
generator = pipeline('text-generation', model='gpt2')
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [308]:
def create_sentence(sentence):
    words = sentence.split()
    spilt_num_word = max(1, len(words) // 2)
    mask_word = random.sample(range(len(words)), spilt_num_word)

    mask_sentence = []

    for i in range(len(words)):
        if i not in mask_word:
            mask_sentence.append(words[i])
        else:
            mask_sentence.append(pipe.tokenizer.mask_token)
    mask_sentence = " ".join(mask_sentence)

    predictions = pipe(mask_sentence)

    new_words = words.copy()

    for i, index in enumerate(mask_sentence):
        if i < len(predictions) and 'token_str' in predictions[i]:
            new_words = predictions[i]['token_str']

    new_sentence = " ".join(new_words)

    generate = generator(new_sentence, max_length=100, num_return_sequences=1, temperature=0.6, do_sample=True)
    generate_sentence = generate[0]['generated_text']

    return generate_sentence

In [309]:
sentence = "After your workout, remember to focus on maintaining a good water balance."

loop_num = 100
similar_status = 0.85
create_new_sentences = ()

for i in range(loop_num):
    create_new_sentence = create_sentence(sentence)

    if create_new_sentence not in create_new_sentence:
        create_new_sentences.add(create_new_sentence)

    sentence_emb = sentence_model.encode(sentence, convert_to_tensor=True)
    new_sentence_emb = sentence_model.encode(create_new_sentence, convert_to_tensor=True)

    similarity = util.pytorch_cos_sim(sentence_emb, new_sentence_emb)[0][0].item()

    if similarity > similar_status:
        break

print(f"Sentence: {create_new_sentence}\nSimilarity: {similarity}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentence: After your workout, remember to focus on maintaining a good water balance.

A good water balance is one that gives you a good chance to get the best out of your workout.

If you're doing water work, you'll want to put on some water.

A good water balance also means that you'll be working out at least 3 times a week.

If you're doing water work twice a week, do it at once.

If you're doing water
Similarity: 0.857189953327179
